In [1]:
# If you have compiled Mitsuba 3 yourself, you will need to specify the path
# to the compilation folder
# import sys
# sys.path.insert(0, '<mitsuba-path>/mitsuba3/build/python')
import mitsuba as mi
# To set a variant, you need to have set it in the mitsuba.conf file
# https://mitsuba.readthedocs.io/en/latest/src/key_topics/variants.html
print(mi.variants())
mi.set_variant('cuda_ad_rgb')

import mitransient as mitr

print('Using mitsuba version:', mi.__version__)
print('Using mitransient version:', mitr.__version__)

['scalar_rgb', 'scalar_spectral', 'scalar_spectral_polarized', 'llvm_ad_rgb', 'llvm_ad_mono', 'llvm_ad_mono_polarized', 'llvm_ad_spectral', 'llvm_ad_spectral_polarized', 'cuda_ad_rgb', 'cuda_ad_mono', 'cuda_ad_mono_polarized', 'cuda_ad_spectral', 'cuda_ad_spectral_polarized']
Using mitsuba version: 3.7.0
Using mitransient version: 1.2.0


In [2]:
import drjit as dr
import numpy as np

def get_rays_mitsuba(H, W, fx, fy, cx, cy, c2w_matrix):
    """
    Generate camera rays using DrJit/Mitsuba instead of PyTorch.
    
    Args:
        H, W: Image height and width
        fx, fy: Focal lengths
        cx, cy: Principal point coordinates
        c2w_matrix: Camera-to-world transformation matrix (4x4 numpy array)
    
    Returns:
        origins: Ray origins (H, W, 3) numpy array
        viewdirs: Ray directions (H, W, 3) numpy array
    """
    # Create pixel coordinates
    num_pixels = H * W
    idx = np.arange(num_pixels)
    x = idx % W
    y = idx // W
    
    # Convert to image coordinates with pixel centers
    # Using the same convention as the torch code: (x - cx + 0.5) / fx
    pixel_x = (x - cx + 0.5) / fx
    pixel_y = (y - cy + 0.5) / fy * -1.0  # Flip Y axis
    
    # Create direction vectors in camera space
    # Format: [pixel_x, pixel_y, -1.0] (pointing into the scene)
    dirs_camera = np.stack([pixel_x, pixel_y, np.full(num_pixels, -1.0)], axis=-1)
    
    # Extract rotation and translation from c2w matrix
    rotation = c2w_matrix[:3, :3]
    translation = c2w_matrix[:3, 3]
    
    # Transform directions to world space
    directions = dirs_camera @ rotation.T  # (num_pixels, 3) @ (3, 3).T
    
    # Normalize directions
    directions = directions / np.linalg.norm(directions, axis=-1, keepdims=True)
    
    # Broadcast camera origin to all rays
    origins = np.broadcast_to(translation, (num_pixels, 3))
    
    # Reshape to image dimensions
    origins = origins.reshape(H, W, 3)
    viewdirs = directions.reshape(H, W, 3)
    
    # Negate viewdirs to match torch convention
    viewdirs = -1 * viewdirs
    
    return origins, viewdirs

d = mitr.cornell_box()

In [3]:
import numpy as np

# Image dimensions
H, W = 255, 255
fov = 0.69097585

# Compute camera intrinsics
fx = (W / 2.0) / np.tan(fov / 2.0)
fy = fx
cx = float(W) / 2.0
cy = float(H) / 2.0

# Get camera-to-world matrix from the sensor
c2w_transform = d['sensor']['to_world']
c2w_matrix = np.array(c2w_transform.matrix)

# Generate rays using Mitsuba/DrJit approach
origins, rays = get_rays_mitsuba(H, W, fx, fy, cx, cy, c2w_matrix)

print(f"Origins shape: {origins.shape}")
print(f"Rays shape: {rays.shape}")
print(f"Sample ray direction at (0,0): {rays[0, 0]}")
print(f"Sample ray origin at (0,0): {origins[0, 0]}")

Origins shape: (255, 255, 3)
Rays shape: (255, 255, 3)
Sample ray direction at (0,0): [-0.31976103 -0.31976101 -0.89191131]
Sample ray origin at (0,0): [0.  0.  3.9]


In [4]:
# Create parametric projector with Gaussian spots using the new ConfocalProjector emitter
import numpy as np

# ==== CONFIGURABLE PARAMETERS ====
grid_rows = 1  # Number of spots along Y
grid_cols = 3  # Number of spots along X
sigma = 0.001   # Standard deviation in normalized coordinates [-1, 1]
spacing_mode = 'uniform'  # 'uniform' or 'random'
random_seed = 42  # Only used if spacing_mode='random'
base_intensity = 100000.0  # Base intensity (will be divided by num_spots)
base_spp = 10000  # Base samples per pixel (will be multiplied by num_spots)
max_rejection_samples = 4  # Max rejection sampling iterations for out-of-FOV samples
fov = 0.55

num_spots = grid_rows * grid_cols
intensity = base_intensity / num_spots  # Divide intensity by number of spots
spp = base_spp * num_spots  # Scale samples by number of spots

print(f"Parametric projector configuration:")
print(f"  Grid size: {grid_rows} rows x {grid_cols} cols = {num_spots} spots")
print(f"  Gaussian sigma (normalized): {sigma}")
print(f"  Intensity per spot: {intensity} (base: {base_intensity})")
print(f"  Samples per pixel: {spp} (base: {base_spp})")
print(f"  Max rejection samples: {max_rejection_samples}")

Parametric projector configuration:
  Grid size: 1 rows x 3 cols = 3 spots
  Gaussian sigma (normalized): 0.001
  Intensity per spot: 33333.333333333336 (base: 100000.0)
  Samples per pixel: 30000 (base: 10000)
  Max rejection samples: 4


In [ ]:
# Remove keys from a list
keys_to_remove = ['light', 'integrator']  # Example: remove these plugins if present
for k in keys_to_remove:
    d.pop(k, None)

# Update film parameters for batch rendering
d['sensor']['film']['temporal_bins'] = 1000
d['sensor']['film']['width'] = W
d['sensor']['film']['height'] = H
print(d['sensor'])

# Get camera to_world transform directly from the sensor
# This is already a Transform4f with rotation and position
camera_to_world = d['sensor']['to_world']

print(f"\nCamera to_world transform:")
print(camera_to_world)

# Create the ConfocalProjector emitter using grid mode
# Pass the camera transform as the projector frame for non-confocal mode
from mitransient.emitters.confocal_projector import ConfocalProjector

projector = mi.load_dict({
    "type": "confocal_projector",
    "grid_rows": grid_rows,
    "grid_cols": grid_cols,
    "grid_sigma": sigma,
    "grid_intensity": intensity,
    "grid_spacing": spacing_mode,
    "fov": fov,
    "is_confocal": False,  # Set to False to use static projector frame
    "frame": camera_to_world,  # Pass full 4x4 Transform4f (rotation + position)
    "max_rejection_samples": max_rejection_samples,
})

print(f"\nConfocal projector created:")
print(projector.to_string())

# Setup integrator with reference to the projector
integrator = mi.load_dict({
    "type": "transient_path",
    "temporal_filter": "gaussian",
    "gaussian_stddev": 4.0,
    "use_nlos_only": False,
    "camera_unwarp": True,
    "confocal_projector": projector,
})
d['integrator'] = integrator

print(f"\nScene created successfully!")
print(f"Using parametric projector: {grid_rows}x{grid_cols} grid, sigma={sigma}")
print(f"Number of spots: {num_spots}")
print(f"Samples per pixel: {spp}")
print(f"Projector mode: {'Confocal (dynamic)' if projector.is_confocal else 'Static (camera-aligned)'}")

scene = mi.load_dict(d)
data_steady, data_transient = mi.render(scene, spp=spp)

In [ ]:
# Plot the computed transient image as a video
data_transient_clipped = dr.clip(data_transient, 0.0, 1.0)
data_transient_tonemapped = mitr.vis.tonemap_transient(data_transient_clipped)

mitr.vis.save_video(
    'results/cornell_confocal_output.mp4',
    data_transient_tonemapped,
    axis_video=2,
)

0.103059374


In [ ]:
# Plot some frames of the computed transient image
import matplotlib.pyplot as plt

data_transient_tonemapped[data_transient_tonemapped > 1] = 1

plt.subplot(1, 2, 1)
plt.axis("off")
plt.imshow(data_transient_tonemapped[:, :, 100])  # frame 100 (video has 300 frames)
plt.subplot(1, 2, 2)
plt.axis("off")
plt.imshow(data_transient_tonemapped[:, :, 140])  # frame 140 (video has 300 frames)
plt.show()